Na etapa de gerar relatorio encontrei a necessidade de criar colunas adicionais na tabela entregas, id_motorista e motivo_cancelamento 

A baixo essa resolução desse problema 

In [14]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/victorsabino36/Documents/projetos_dev/credenciais/bigquery_cre.json"
from google.cloud import bigquery
import random
import pandas as pd
dataset_id = "mythical-lyceum-450717-f6.transporte_log"
client = bigquery.Client()
import numpy as np

In [7]:
try:
    
    # Defina a consulta SQL para obter os IDs dos motoristas
    query_motoristas = """
    SELECT id_motorista
    FROM `mythical-lyceum-450717-f6.transporte_log.motoristas`
    """

    # Executando a consulta e carregando os resultados em um DataFrame
    motoristas_df = client.query(query_motoristas).to_dataframe()

    # Visualizar os motoristas
    print(motoristas_df.head())

except Exception as e:
    print(f"Erro: {e}")

   id_motorista
0             6
1             7
2             9
3            10
4            11


In [11]:
try:

    # Defina a consulta SQL para obter as entregas
    query_entregas = """
    SELECT *
    FROM `mythical-lyceum-450717-f6.transporte_log.entregas`
    """

    # Executando a consulta e carregando os resultados em um DataFrame
    entregas_df = client.query(query_entregas).to_dataframe()

    # Adiciona a coluna `id_motorista` de forma aleatória, selecionando IDs da tabela motoristas
    entregas_df['id_motorista'] = [random.choice(motoristas_df['id_motorista'].tolist()) for _ in range(len(entregas_df))]

    # Visualizar as primeiras linhas do DataFrame de entregas com a nova coluna
    print(entregas_df.head())

except Exception as e:
    print(f"Erro: {e}")

   id_entrega  data_envio data_entrega     status             cliente  \
0          74  2025-02-18   2025-02-19  Cancelado  Christopher Holmes   
1          62  2025-01-04   2025-01-14  Cancelado        Michael Neal   
2          81  2025-01-19   2025-01-29  Cancelado      Yolanda Wright   
3          16  2025-01-17   2025-01-19  Cancelado       Steven Butler   
4          32  2025-01-09   2025-01-11  Cancelado    Richard Campbell   

                cidade estado    valor tempo_entrega  id_motorista  
0           Smithshire     FL  3906.10        1 dias             6  
1         West Antonio     WV  4879.29       10 dias            20  
2  South Justinchester     ND  2915.24       10 dias             6  
3          South Carla     IA  3187.91        2 dias            29  
4           Port Tyler     HI  1268.35        2 dias            13  


In [ ]:
# Defina o dataset e a tabela no BigQuery
dataset_id = 'mythical-lyceum-450717-f6.transporte_log'  # Substitua com o seu dataset
table_id = 'entregas'  # Nome da tabela de destino

# Carregar o DataFrame no BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # Isso adiciona os dados sem sobrescrever a tabela existente
    # Se preferir sobrescrever a tabela, use "WRITE_TRUNCATE"
)

# Carregar o DataFrame
job = client.load_table_from_dataframe(
    entregas_df, f"{dataset_id}.{table_id}", job_config=job_config
)

# Espera a finalização do job
job.result()

# Verificando o status da inserção
print(f"Dados carregados com sucesso para {dataset_id}.{table_id}")


Dados carregados com sucesso para mythical-lyceum-450717-f6.transporte_log.entregas


In [6]:
# Defina o ID da tabela de entregas
entregas_table_id = "mythical-lyceum-450717-f6.transporte_log.entregas"

# Carregar os dados atualizados de volta para a tabela de entregas no BigQuery
job = client.load_table_from_dataframe(entregas_df, entregas_table_id)

# Aguardar o término do job de carregamento
job.result()

print("Tabela de entregas atualizada com sucesso no BigQuery!")


Tabela de entregas atualizada com sucesso no BigQuery!


In [31]:
try:

    # Defina a consulta SQL para obter as entregas
    query_entregas = """
    SELECT *
    FROM `mythical-lyceum-450717-f6.transporte_log.entregas`
    """

    # Executando a consulta e carregando os resultados em um DataFrame
    entregas_df = client.query(query_entregas).to_dataframe()

except Exception as e:
    print(f"Erro: {e}")

In [32]:
# Definir os possíveis motivos de cancelamento
motivos = ["Cliente ausente", "Endereço incorreto", "Problema no veículo"]

# Garantir que a coluna 'status' não tem espaços extras ou valores inesperados
entregas_df["status"] = entregas_df["status"].str.strip().str.lower()

# Criar uma nova coluna com valores padrão ""
entregas_df["motivo_cancelamento"] = ""

# Criar uma máscara para identificar os cancelados
mask = entregas_df["status"] == "cancelado"

# Aplicar motivos aleatórios apenas para os cancelados
if mask.sum() > 0:  # Verifica se há cancelamentos para evitar erro
    entregas_df.loc[mask, "motivo_cancelamento"] = np.random.choice(motivos, size=mask.sum(), replace=True)

# Verificar os dados
print(entregas_df[["status", "motivo_cancelamento"]].value_counts())


status       motivo_cancelamento
pendente                            64
entregue                            60
cancelado    Problema no veículo    27
             Cliente ausente        27
             Endereço incorreto     24
em trânsito                          2
Name: count, dtype: int64


In [35]:
entregas_df.head(10)

,id_entrega,data_envio,data_entrega,status,cliente,cidade,estado,valor,tempo_entrega,id_motorista,motivo_cancelamento
0,1,2024-10-10,NaT,None,None,None,None,NaN,None,34,
1,2,2024-10-10,NaT,None,None,None,None,NaN,None,14,
2,1,2024-10-10,NaT,None,None,None,None,NaN,None,19,
3,2,2024-10-10,NaT,None,None,None,None,NaN,None,25,
4,2,NaT,NaT,em trânsito,Maria Souza,Rio de Janeiro,RJ,250.00,None,9,
5,2,NaT,NaT,em trânsito,Maria Souza,Rio de Janeiro,RJ,250.00,None,12,
6,50,2025-01-22,2025-01-23,pendente,Emily Robinson,Bradleymouth,AL,2069.04,1 dias,27,
7,50,2025-01-22,2025-01-23,pendente,Emily Robinson,Bradleymouth,AL,2069.04,1 dias,202,
8,97,2025-02-13,2025-02-14,pendente,Mary Chang,West Jacob,TX,2477.83,1 dias,17,
9,97,2025-02-13,2025-02-14,pendente,Mary Chang,West Jacob,TX,2477.83,1 dias,13,


In [36]:
# Defina o dataset e a tabela no BigQuery
dataset_id = 'mythical-lyceum-450717-f6.transporte_log'  # Substitua com o seu dataset
table_id = 'entregas'  # Nome da tabela de destino

# Carregar o DataFrame no BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # Isso adiciona os dados sem sobrescrever a tabela existente
    # Se preferir sobrescrever a tabela, use "WRITE_TRUNCATE"
)

# Carregar o DataFrame
job = client.load_table_from_dataframe(
    entregas_df, f"{dataset_id}.{table_id}", job_config=job_config
)

# Espera a finalização do job
job.result()

# Verificando o status da inserção
print(f"Dados carregados com sucesso para {dataset_id}.{table_id}")


Dados carregados com sucesso para mythical-lyceum-450717-f6.transporte_log.entregas
